# <b>DISCUSSÃO DOS RESULTADOS</b>

## <b>Visão Geral</b>

O código fornecido utiliza um modelo de aprendizado profundo pré-treinado para classificar imagens de caminhões em categorias específicas. O modelo parece estar treinado para identificar caminhões pequenos, caminhões grandes e caminhões danificados. A classificação é baseada em um limite de confiança, e o código imprime a classe prevista e a pontuação de confiança.

## <b>Visão Geral do Código</b>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install opencv-python

import cv2

In [ ]:
!pip install Pillow

In [ ]:
from keras.models import load_model
import numpy as np

In [ ]:
from PIL import Image, ImageOps  # Instale pillow em vez de PIL

In [ ]:
import numpy as np


In [ ]:
# Desativar a notação científica para clareza
np.set_printoptions(suppress=True)

## 1. <b>Carregamento do Modelo:</b>
 O modelo treinado é carregado de um arquivo .h5
 , e os rótulos de classe são carregados de um arquivo de texto.

In [ ]:
# Carregar o modelo
model = load_model("/content/drive/MyDrive/CAMINHÕES/keras_model.h5", compile=False)

In [ ]:
# Carregar os rótulos
class_names = open("/content/drive/MyDrive/CAMINHÕES/labels.txt", "r").readlines()

## 2. <b>Preparação da Imagem:</b>
Uma imagem é carregada, redimensionada para 224x224 pixels e convertida em uma matriz numpy. A matriz é então normalizada para atender aos requisitos de entrada do modelo.

In [ ]:
# Criar o array da forma certa para alimentar o modelo keras
# O 'comprimento' ou número de imagens que você pode colocar no array é
# determinado pela primeira posição na tupla de forma, neste caso 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [ ]:
# Substitua isso pelo caminho da sua imagem
image = Image.open("/content/drive/MyDrive/CAMINHÕES/GRANDE.jpg").convert("RGB")

In [ ]:
# redimensionando a imagem para ter pelo menos 224x224 e depois cortando do centro
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

In [ ]:
# transformar a imagem em um array numpy
image_array = np.asarray(image)

In [ ]:
# Normalizar a imagem
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

In [ ]:
# Carregar a imagem no array
data[0] = normalized_image_array

In [ ]:
confidence_threshold = 0.70

## 3. <b>Predição:</b>
A imagem pré-processada é alimentada ao modelo, e as previsões são obtidas.

In [ ]:
# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

## 4. <b>Impressão dos Resultados: </b>
Com base na previsão de maior confiança acima de um limiar especificado, o código imprime a classe prevista e a pontuação de confiança.

In [ ]:
# Verificar as previsões e imprimir resultados
if prediction[0][0] > confidence_threshold:
    print("Este é um caminhão de pequeno porte. Envie o guincho Plataforma.")
    print("Class:", class_name[0:], end="")
    print("Pontuação de confiança:  {:.2f}".format(confidence_score))
elif prediction[0][2] > confidence_threshold:
    print("Este é um caminhão de grande porte. Envie o guincho Lança.")
    print("Class:", class_name[2:], end="")
    print("Pontuação de confiança:  {:.2f}".format(confidence_score))
elif prediction[0][3] > confidence_threshold:
    print("Este caminhão está danificado. Envie o guincho Plataforma.")
    print("Class:", class_name[3:], end="")
    print("Pontuação de confiança:  {:.2f}".format(confidence_score))
else:
    print("Imagem não identificada. Tente tirar a foto de outro ângulo.")


1/1 [==============================] - 3s 3s/step
Este é um caminhão de grande porte. Envie o guincho Lança.
Class: Caminhões Grandes
Pontuação de confiança:  0.71


## <b>Pontos de Discussão</b>
#### 1.  Limite de Confiança
- O <b>"confidence_threshold"</b> está definido como 0.70, o que significa que previsões com pontuação de confiança abaixo desse valor são consideradas não confiáveis e não são impressas.
- A escolha desse limite influencia a compensação entre falsos positivos e falsos negativos. Pode ser necessário ajustá-lo dependendo dos requisitos da aplicação.

#### 2. Rótulos de Classe
- Os nomes das classes são extraídos do arquivo de rótulos carregado. Garantir que esses rótulos estejam alinhados com os dados de treinamento é importante para uma interpretação correta.

#### 3. Tratamento de Previsões Incertas
- A implementação atual não lida explicitamente com casos em que as pontuações de confiança para várias classes são próximas. Considerar tais casos pode evitar classificações incorretas.

#### 5. Consideração da Qualidade da Imagem
- O código menciona que, se o limite de confiança não for atingido, a imagem não é identificada. Considerar fornecer feedback ou instruções ao usuário para casos em que a qualidade ou ângulo da imagem não são ideais.